In [ ]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import pandas as pd
import pdo_functions
import importlib
importlib.reload(pdo_functions)

In [ ]:
# Open datasets: want CAPE, S06, NDSEV, 6 km wind and surface wind
cape = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/wrf_cape/era5_daily_max_mucape_*s.nc', combine='nested', concat_dim='year')['__xarray_dataarray_variable__']
s06 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/era5_s06_*s.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']

u10 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/u10/*s/e5.oper.an.sfc.128_165_10u.*.nc', combine='nested', concat_dim='time')['VAR_10U']
v10 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/v10/*s/e5.oper.an.sfc.128_166_10v.*.nc', combine='nested', concat_dim='time')['VAR_10V']

u6k = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/u_6k/*.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']
v6k = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/v_6k/*.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']

ndsev = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/ndsev/diff_trapp_ndsev_mam_1940-2024.nc')['__xarray_dataarray_variable__']

In [ ]:
# Open SST data and compute the PDO and Nino3.4
sst = xr.open_dataset('/hurrell-scratch2/ivyglade/pdo/HadISST_sst.nc')['sst']

pdo = pdo_functions.pdo_from_hadisst(sst, 1870, 2024)

pdo['time'] = sst['time']

nino_34 = pdo_functions.oni_moving_base(sst)

In [ ]:
# Convert pdo to xarray
pdo_xr = xr.DataArray(pdo, coords={'time':sst['time']}, dims=['time'])

# Subset only 1940-2024
pdo_1940_2024 = pdo_xr.sel(time=pdo_xr.time.dt.year.isin(np.arange(1940, 2025, 1)))
# nino_34_1940_2024 = nino_34_std.sel(time=nino_34_std.time.dt.year.isin(np.arange(1940, 2025, 1)))

# only MAM and take seasonal averages
pdo_1940_2024_mam = pdo_1940_2024.sel(time=pdo_1940_2024.time.dt.month.isin([3, 4, 5]))#.resample(time='1Y').mean()
# nino_34_1940_2024_mam = nino_34_1940_2024.sel(time=nino_34_1940_2024.time.dt.month.isin([3, 4, 5]))#.resample(time='1Y').mean()

# aligning time arrays with that of CAPE
# pdo_1940_2024_mam['time'] = cape_east_mam_anoms['time']
# nino_34_1940_2024_mam['time'] = cape_east_mam_anoms['time']

# s06_east_mam_anoms['time'] = cape_east_mam_anoms['time']

In [ ]:
pdo_daily = pd.Series(pdo_1940_2024_mam.values, index=s06_conus_mam.resample(time='ME').mean().dropna(dim='time')['time']).reindex(s06_conus_mam['time'], method='bfill')

In [ ]:
pdo_daily_xr = xr.DataArray(pdo_daily.values, coords={'time': s06_conus_mam['time']}, dims=['time'])

In [ ]:
# subset only MAM
cape_mam = cape.sel(date=cape.date.where(
                         cape.date.str.startswith('03-') |
                         cape.date.str.startswith('04-') |
                         cape.date.str.startswith('05-'),
                         drop=True)).load()

In [ ]:
# flatten to have a single time dimension
cape_mam_stack = cape_mam.stack(time=('year', 'date'))

# Convert time to datetime
year = cape_mam_stack['time'].get_index('time').get_level_values('year')
date = cape_mam_stack['time'].get_index('time').get_level_values('date')

datetime = pd.to_datetime(year.astype(str) + '-' + date.astype(str))

cape_mam_stack = cape_mam_stack.assign_coords(time=datetime)

In [ ]:
# Subsetting the S06 dataset
# Conus
s06_conus = s06.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

# MAM only
s06_conus_mam = s06_conus.sel(time=s06_conus.time.dt.month.isin([3, 4, 5])).transpose('latitude', 'longitude', 'time').load()

In [ ]:
# Need to compute monthly means
ndsev_monthly = np.zeros((85, 101, 237))
for i in range(85):
    # Load in one year of data
    ndsev_monthly[i] = ndsev.isel(time=slice(i*92, (i+1)*92)).transpose('time', 'latitude', 'longitude').values.sum(axis=0)

In [ ]:
# Subsetting wind data
# CONUS
u10_conus = u10.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
v10_conus = v10.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
u6k_conus = u6k.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
v6k_conus = v6k.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

# MAM only
u10_conus_mam = u10_conus.sel(time=u10_conus.time.dt.month.isin([3, 4, 5])).load()
v10_conus_mam = v10_conus.sel(time=v10_conus.time.dt.month.isin([3, 4, 5])).load()
u6k_conus_mam = u6k_conus.sel(time=u6k_conus.time.dt.month.isin([3, 4, 5])).load()
v6k_conus_mam = v6k_conus.sel(time=v6k_conus.time.dt.month.isin([3, 4, 5])).load()

In [ ]:
# Compute wind speed
wind_sfc = np.sqrt(u10_conus_mam**2 + v10_conus_mam**2)
wind_6k = np.sqrt(u6k_conus_mam**2 + v6k_conus_mam**2)

In [ ]:
# Align time dimensions
cape_mam_stack['time'] = s06_conus_mam['time']

In [ ]:
# CAPES06
capes06 = cape_mam_stack * s06_conus_mam

In [ ]:
capes06_above_thresh = xr.where(capes06 > 10000, 1, 0)
cape_and_s06_above_thresh = xr.where((cape_mam_stack > 100) & (s06_conus_mam > 5), 1, 0)
wind_sfc_above_thresh = xr.where(wind_sfc > 5, 1, 0)
wind_6k_big_above_thresh = xr.where(wind_6k > wind_sfc, 1, 0)

In [ ]:
# Leave one out composites
no_cape_above_thresh = xr.where((capes06 > 10000) & (s06_conus_mam > 5) & (wind_sfc > 5) & (wind_6k > wind_sfc), 1, 0)
no_s06_above_thresh = xr.where((capes06 > 10000) & (cape_mam_stack > 100) & (wind_sfc > 5) & (wind_6k > wind_sfc), 1, 0)
no_capes06_above_thresh = xr.where((s06_conus_mam > 5) & (cape_mam_stack > 100) & (wind_sfc > 5) & (wind_6k > wind_sfc), 1, 0)
no_wind_sfc_above_thresh = xr.where((capes06 > 10000) & (cape_mam_stack > 100) & (s06_conus_mam > 5) & (wind_6k > wind_sfc), 1, 0)
no_wind_6k_big_above_thresh = xr.where((capes06 > 10000) & (cape_mam_stack > 100) & (s06_conus_mam > 5) & (wind_sfc > 5), 1, 0)

all_together = xr.where((capes06 > 10000) & (s06_conus_mam > 5) & (wind_sfc > 5) & (wind_6k > wind_sfc) & (cape_mam_stack > 100), 1, 0)

In [ ]:
def calc_ndsev_trapp_diff(cape, s06, wind_6k, wind_sfc):
    cond = (
        (cape * s06 > 10000) &
        (cape > 100) &
        (s06 > 5) &
        (wind_6k > wind_sfc) &
        (wind_sfc > 5))
    return cond.astype(int)

In [ ]:
function = calc_ndsev_trapp_diff(cape_mam_stack, s06_conus_mam, wind_6k, wind_sfc)

In [ ]:
capes06_day_per_year_above_thresh = capes06_above_thresh.groupby('time.year').sum()
cape_and_s06_day_per_year_above_thresh = cape_and_s06_above_thresh.groupby('time.year').sum()
wind_sfc_day_per_year_above_thresh = wind_sfc_above_thresh.groupby('time.year').sum()
wind_6k_big_day_per_year_above_thresh = wind_6k_big_above_thresh.groupby('time.year').sum()

no_cape_dpy_above_thresh = no_cape_above_thresh.groupby('time.year').sum()
no_s06_dpy_above_thresh = no_s06_above_thresh.groupby('time.year').sum()
no_capes06_dpy_above_thresh = no_capes06_above_thresh.groupby('time.year').sum()
no_wind_sfc_dpy_above_thresh = no_wind_sfc_above_thresh.groupby('time.year').sum()
no_wind_6k_big_dpy_above_thresh = no_wind_6k_big_above_thresh.groupby('time.year').sum()
all_together_dpy = all_together.groupby('time.year').sum()
function_dpy = function.groupby('time.year').sum()

In [ ]:
all_together_dpy.mean(dim=('latitude', 'longitude')).plot()
function_dpy.mean(dim=('latitude', 'longitude')).plot(linestyle='--')

In [ ]:
pdo_mam_mean = pdo_1940_2024_mam.groupby('time.year').mean()

In [ ]:
capes06_day_per_year_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, capes06_day_per_year_above_thresh, np.nan).dropna(dim='year')
capes06_day_per_year_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, capes06_day_per_year_above_thresh, np.nan).dropna(dim='year')

cape_and_s06_day_per_year_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, cape_and_s06_day_per_year_above_thresh, np.nan).dropna(dim='year')
cape_and_s06_day_per_year_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, cape_and_s06_day_per_year_above_thresh, np.nan).dropna(dim='year')

wind_sfc_day_per_year_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, wind_sfc_day_per_year_above_thresh, np.nan).dropna(dim='year')
wind_sfc_day_per_year_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, wind_sfc_day_per_year_above_thresh, np.nan).dropna(dim='year')

wind_6k_big_day_per_year_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, wind_6k_big_day_per_year_above_thresh, np.nan).dropna(dim='year')
wind_6k_big_day_per_year_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, wind_6k_big_day_per_year_above_thresh, np.nan).dropna(dim='year')

In [ ]:
no_cape_dpy_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, no_cape_dpy_above_thresh, np.nan).dropna(dim='year')
no_cape_dpy_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, no_cape_dpy_above_thresh, np.nan).dropna(dim='year')

no_s06_dpy_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, no_s06_dpy_above_thresh, np.nan).dropna(dim='year')
no_s06_dpy_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, no_s06_dpy_above_thresh, np.nan).dropna(dim='year')

no_capes06_dpy_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, no_capes06_dpy_above_thresh, np.nan).dropna(dim='year')
no_capes06_dpy_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, no_capes06_dpy_above_thresh, np.nan).dropna(dim='year')

no_wind_sfc_dpy_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, no_wind_sfc_dpy_above_thresh, np.nan).dropna(dim='year')
no_wind_sfc_dpy_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, no_wind_sfc_dpy_above_thresh, np.nan).dropna(dim='year')

no_wind_6k_big_dpy_above_thresh_pos_pdo = xr.where(pdo_mam_mean > 0, no_wind_6k_big_dpy_above_thresh, np.nan).dropna(dim='year')
no_wind_6k_big_dpy_above_thresh_neg_pdo = xr.where(pdo_mam_mean < 0, no_wind_6k_big_dpy_above_thresh, np.nan).dropna(dim='year')

In [ ]:
all_together_dpy_de = pdo_functions.detrend_dim(all_together_dpy, 'year', 2)
function_dpy_de = pdo_functions.detrend_dim(function_dpy, 'year', 2)

In [ ]:
all_together_dpy_pos_pdo = xr.where(pdo_mam_mean > 0, all_together_dpy_de, np.nan).dropna(dim='year')
all_together_dpy_neg_pdo = xr.where(pdo_mam_mean < 0, all_together_dpy_de, np.nan).dropna(dim='year')

function_dpy_pos_pdo = xr.where(pdo_mam_mean > 0, function_dpy_de, np.nan).dropna(dim='year')
function_dpy_neg_pdo = xr.where(pdo_mam_mean < 0, function_dpy_de, np.nan).dropna(dim='year')

In [ ]:
fig, ax = plt.subplots(2, 2, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], \
      ax[1, 0], ax[1, 1]]
#       ax[2, 0], ax[2, 1], \
#       ax[3, 0], ax[3, 1], \
#       ax[4, 0], ax[4, 1]]

bounds = [-2, -1.8, -1.6, -1.4, -1.2, -1, -0.8, -0.6, -0.4, -0.2, -0.02, 0.02, 0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2]

norm = c.BoundaryNorm(bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(4):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0.25)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')

ax[0].pcolormesh(capes06['longitude'], capes06['latitude'], all_together_dpy_pos_pdo.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
ax[1].pcolormesh(capes06['longitude'], capes06['latitude'], all_together_dpy_neg_pdo.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

ax[2].pcolormesh(capes06['longitude'], capes06['latitude'], function_dpy_pos_pdo.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
ax[3].pcolormesh(capes06['longitude'], capes06['latitude'], function_dpy_neg_pdo.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

# ax[2].pcolormesh(capes06['longitude'], capes06['latitude'], no_s06_dpy_above_thresh_pos_pdo.mean(dim='year') - no_s06_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
# ax[3].pcolormesh(capes06['longitude'], capes06['latitude'], no_s06_dpy_above_thresh_neg_pdo.mean(dim='year') - no_s06_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

# ax[4].pcolormesh(capes06['longitude'], capes06['latitude'], no_capes06_dpy_above_thresh_pos_pdo.mean(dim='year') - no_capes06_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
# ax[5].pcolormesh(capes06['longitude'], capes06['latitude'], no_capes06_dpy_above_thresh_neg_pdo.mean(dim='year') - no_capes06_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

# ax[6].pcolormesh(capes06['longitude'], capes06['latitude'], no_wind_sfc_dpy_above_thresh_pos_pdo.mean(dim='year') - no_wind_sfc_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
# ax[7].pcolormesh(capes06['longitude'], capes06['latitude'], no_wind_sfc_dpy_above_thresh_neg_pdo.mean(dim='year') - no_wind_sfc_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

# ax[8].pcolormesh(capes06['longitude'], capes06['latitude'], no_wind_6k_big_dpy_above_thresh_pos_pdo.mean(dim='year') - no_wind_6k_big_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)
# ax[9].pcolormesh(capes06['longitude'], capes06['latitude'], no_wind_6k_big_dpy_above_thresh_neg_pdo.mean(dim='year') - no_wind_6k_big_dpy_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), cmap='cmr.fusion_r', norm=norm)

In [ ]:
fig, ax = plt.subplots(3, 2, subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=36.5)))

ax = [ax[0, 0], ax[0, 1], \
      ax[1, 0], ax[1, 1], \
      ax[2, 0], ax[2, 1]]

bounds = np.arange(-5, 5.5, 0.5)

norm = c.BoundaryNorm(bounds, plt.get_cmap('cmr.fusion_r').N)

for i in range(6):
    ax[i].coastlines(lw=0.25, color='xkcd:gunmetal')
    ax[i].spines['geo'].set_edgecolor('xkcd:gunmetal')
    ax[i].spines['geo'].set_linewidth(0.25)
    ax[i].add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')

ax[0].pcolormesh(capes06['longitude'], capes06['latitude'], cape_and_s06_day_per_year_above_thresh_pos_pdo.mean(dim='year') - cape_and_s06_day_per_year_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')
ax[1].pcolormesh(capes06['longitude'], capes06['latitude'], cape_and_s06_day_per_year_above_thresh_neg_pdo.mean(dim='year') - cape_and_s06_day_per_year_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')

ax[2].pcolormesh(capes06['longitude'], capes06['latitude'], wind_sfc_day_per_year_above_thresh_pos_pdo.mean(dim='year') - wind_sfc_day_per_year_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')
ax[3].pcolormesh(capes06['longitude'], capes06['latitude'], wind_sfc_day_per_year_above_thresh_neg_pdo.mean(dim='year') - wind_sfc_day_per_year_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')

ax[4].pcolormesh(capes06['longitude'], capes06['latitude'], wind_6k_big_day_per_year_above_thresh_pos_pdo.mean(dim='year') - wind_6k_big_day_per_year_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')
ax[5].pcolormesh(capes06['longitude'], capes06['latitude'], wind_6k_big_day_per_year_above_thresh_neg_pdo.mean(dim='year') - wind_6k_big_day_per_year_above_thresh.mean(dim='year'), transform=ccrs.PlateCarree(), norm=norm, cmap='cmr.fusion_r')

In [ ]:
day_per_year_above_thresh.mean(dim='year').max()